In [1]:
import numpy as np
import os
import glog as log
import copy

import open3d as o3d
from open3d import JVisualizer
import pandas as pd

from evaluation.tools.mesh import Mesh
from evaluation.tools.mesh_evaluator import MeshEvaluator

# Rotation matrices:
# East North Up (ENU) frame to Unity's world frame of reference
enu_R_unity = np.array([[1, 0, 0],
                        [0, 0, 1],
                        [0, 1, 0]])
unity_R_enu = np.transpose(enu_R_unity)

# Right Handed frame to Unity's Left Handed frame of reference
righthand_R_lefthand = np.array([[1, 0, 0],
                                 [0, -1, 0],
                                 [0, 0, 1]])
lefthand_R_righthand = np.transpose(righthand_R_lefthand)

In [2]:
# FILL PATHS BELOW
#gt_mesh_path = "/home/tonirv/Downloads/tesse_multiscene_office1_3d_semantic_v5.ply"
#est_mesh_path = "/home/tonirv/Downloads/tesse_semantics_2.ply"

#gt_mesh_path = "/home/tonirv/Code/ROS/flight_goggles_ws/src/voxblox/voxblox_ros/mesh_results/semantic_mesh_tonirv_ld_9118_6487309760727328010.ply"
#est_mesh_path = "/home/tonirv/Code/ROS/flight_goggles_ws/src/voxblox/voxblox_ros/mesh_results/semantic_mesh_tonirv_ld_9118_6487309760727328010.ply"

gt_mesh_path = "/home/yunchang/Downloads/subway.ply"
est_mesh_path = "/home/yunchang/Downloads/uHumans2_subway_s1_00h.ply"

# gt_mesh_path = "/home/yunchang/Desktop/uhumans1/office1_tony.ply"
# est_mesh_path = "/home/yunchang/Desktop/uhumans1/tesse_yunchang_ThinkPad_P72_8018_3440445341542521530.ply"

In [3]:
print("Loading Ground-truth mesh...")
gt_mesh_original = Mesh(gt_mesh_path)
print("Loading Estimated mesh...")
est_mesh_original = Mesh(est_mesh_path)

Loading Ground-truth mesh...
Testing mesh in open3d ...
geometry::TriangleMesh with 3235419 points and 1078473 triangles.
[[-35.02077   -6.990101 -42.77373 ]
 [-35.02077   -6.990101 -43.41461 ]
 [-35.66165   -6.990101 -43.41461 ]
 ...
 [-54.00388  -16.80481   36.92789 ]
 [-58.00388  -16.79861   36.90939 ]
 [-54.00388  -16.79861   36.9094  ]]
[[      0       1       2]
 [      3       4       5]
 [      6       7       8]
 ...
 [3235410 3235411 3235412]
 [3235413 3235414 3235415]
 [3235416 3235417 3235418]]

Loading Estimated mesh...
Testing mesh in open3d ...
geometry::TriangleMesh with 6206230 points and 7157447 triangles.
[[-2.27250e+01  1.52899e+01 -8.42500e+00]
 [-2.27274e+01  1.53250e+01 -8.42500e+00]
 [-2.27250e+01  1.53250e+01 -8.44667e+00]
 ...
 [-3.27250e+01  2.34250e+01  2.29971e-02]
 [-3.27750e+01  2.34219e+01  2.50000e-02]
 [-3.23750e+01  2.33902e+01 -2.50000e-02]]
[[      0       1       2]
 [      3       1       0]
 [      1       4       2]
 ...
 [6206227 6206215 620622

In [4]:
# Transform Meshes to same frame of reference
gt_mesh = copy.deepcopy(gt_mesh_original)
est_mesh = copy.deepcopy(est_mesh_original)

In [5]:
# Align Pointclouds Manually:
# est_mesh.mesh_o3d.translate([0, -5, 0])
# gt_mesh.transform_left(righthand_R_lefthand)
gt_mesh.transform_left(enu_R_unity)

Transforming mesh according to left matrix:
[[1 0 0]
 [0 0 1]
 [0 1 0]]


In [6]:
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.get_render_option().mesh_show_back_face = True
vis.add_geometry(est_mesh.mesh_o3d)
vis.add_geometry(gt_mesh.mesh_o3d)
vis.add_geometry(o3d.geometry.create_mesh_coordinate_frame(size=4))
vis.run()
vis.destroy_window()

In [7]:
NUMBER_OF_SAMPLES=1000000
gt_pcl = o3d.geometry.sample_points_uniformly(gt_mesh.mesh_o3d, NUMBER_OF_SAMPLES)
# Don't sample estimated mesh, just pick vertices, otw you'll be mixing colors...
# est_pcl = o3d.geometry.sample_points_uniformly(est_mesh.mesh_o3d, NUMBER_OF_SAMPLES)
est_pcl = o3d.io.read_point_cloud(est_mesh_path)

In [8]:
# Calculate normals for nice visualization
# THIS COLORS THE PCL?>>>>>????
#o3d.geometry.estimate_normals(
#        est_pcl,
#        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1,
#                                                          max_nn=5))
#o3d.geometry.estimate_normals(
#        gt_pcl,
#        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1,
#                                                          max_nn=5))

In [9]:
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.get_render_option().mesh_show_back_face = True
vis.add_geometry(gt_pcl)
vis.add_geometry(est_pcl)
vis.run()
vis.destroy_window()

In [10]:
# ICP
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])
def draw_correspondences(source, target, correspondences):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    #source_temp.paint_uniform_color([1, 0.706, 0])
    #target_temp.paint_uniform_color([0, 0.651, 0.929])
    o3d.visualization.draw_geometries([source_temp, #target_temp, 
                                       correspondences])
    

In [11]:
# ICP params
ICP_THRESHOLD = 1.5
trans_init = np.asarray([[1.0, 0.0, 0.0, 0.0],
                         [0.0, 1.0, 0.0, 0.0],
                         [0.0, 0.0, 1.0, 0.0],
                         [0.0, 0.0, 0.0, 1.0]])

In [12]:
# Visualize initial registration problem
draw_registration_result(est_pcl, gt_pcl, trans_init)

In [13]:
# Evaluate current fit between pointclouds
evaluation = o3d.registration.evaluate_registration(est_pcl, gt_pcl, ICP_THRESHOLD, trans_init)

In [14]:
print("Initial registration")
print(evaluation)

Initial registration
registration::RegistrationResult with fitness = 1.000000, inlier_rmse = 0.164192, and correspondence_set size of 6206230
Access transformation to get result.


In [15]:
print("Apply point-to-point ICP")
reg_p2p = o3d.registration.registration_icp(
    est_pcl, gt_pcl, ICP_THRESHOLD, trans_init,
    o3d.registration.TransformationEstimationPointToPoint(),
    o3d.registration.ICPConvergenceCriteria(max_iteration = 2000))
correspondences = reg_p2p.correspondence_set

Apply point-to-point ICP


In [16]:
print(reg_p2p)
print("")

print("Transformation is:")
print(reg_p2p.transformation)
print("")

print("Correspondence Set:")
print(reg_p2p.correspondence_set)
print("")

registration::RegistrationResult with fitness = 1.000000, inlier_rmse = 0.164166, and correspondence_set size of 6206230
Access transformation to get result.

Transformation is:
[[ 9.99999991e-01  1.28896061e-04  4.50170245e-05 -3.42781472e-03]
 [-1.28896121e-04  9.99999992e-01  1.32122702e-06 -1.99446779e-03]
 [-4.50168538e-05 -1.32702953e-06  9.99999999e-01 -3.41601256e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

Correspondence Set:
std::vector<Eigen::Vector2i> with 6206230 elements.
Use numpy.asarray() to access data.



In [17]:
# Draw Registration Result
draw_registration_result(est_pcl, gt_pcl, reg_p2p.transformation)

In [18]:
# Draw Only Correspondences
c2c_lines = o3d.geometry.create_line_set_from_point_cloud_correspondences(est_pcl, gt_pcl, correspondences)
o3d.visualization.draw_geometries([c2c_lines])

In [19]:
# Draw PointClouds and Correspondences
draw_correspondences(est_pcl, gt_pcl, c2c_lines)

In [20]:
# Import Semantic Labels
df = pd.read_csv('/home/yunchang/catkin_ws/src/Kimera-Semantics/kimera_semantics_ros/cfg/tesse_multiscene_underground1_segmentation_mapping.csv')
df

,name,red,green,blue,alpha,id
0,Graphics,26,127,81,255,0
1,Intercom,128,29,137,255,0
2,Lamp1,8,10,16,255,11
3,AC_Duct,80,184,210,255,0
4,Graffiti3,12,112,165,255,19
...,...,...,...,...,...,...
372,Tunnel_Service_Door_LOD0,67,53,173,255,19
373,MetalPillar2_SM,25,72,203,255,19
374,Vent_Smoke,138,98,4,255,0
375,SMPL_Female_Autonomous,35,213,234,255,20


In [44]:
normalized_df = copy.deepcopy(df)
normalized_df['normalized_red'] = round(df['red'] / 255, 5)
normalized_df['normalized_green'] = round(df['green'] / 255, 5)
normalized_df['normalized_blue'] = round(df['blue'] / 255, 5)

print(sum(np.isclose(normalized_df['normalized_red'], gt_pcl.colors[0][0])))

1


In [45]:
from hashlib import sha1

from numpy import all, array, uint8


class hashable(object):
    r'''Hashable wrapper for ndarray objects.
        Instances of ndarray are not hashable, meaning they cannot be added to
        sets, nor used as keys in dictionaries. This is by design - ndarray
        objects are mutable, and therefore cannot reliably implement the
        __hash__() method.
        The hashable class allows a way around this limitation. It implements
        the required methods for hashable objects in terms of an encapsulated
        ndarray object. This can be either a copied instance (which is safer)
        or the original object (which requires the user to be careful enough
        not to modify it).
    '''
    def __init__(self, wrapped, tight=False):
        r'''Creates a new hashable object encapsulating an ndarray.
            wrapped
                The wrapped ndarray.
            tight
                Optional. If True, a copy of the input ndaray is created.
                Defaults to False.
        '''
        self.__tight = tight
        self.__wrapped = array(wrapped) if tight else wrapped
        self.__hash = int(sha1(wrapped.view(uint8)).hexdigest(), 16)

    def __eq__(self, other):
        return all(self.__wrapped == other.__wrapped)

    def __hash__(self):
        return self.__hash

    def unwrap(self):
        r'''Returns the encapsulated ndarray.
            If the wrapper is "tight", a copy of the encapsulated ndarray is
            returned. Otherwise, the encapsulated ndarray itself is returned.
        '''
        if self.__tight:
            return array(self.__wrapped)



In [47]:
# No need to hash, we have ids now
red_col = df['red'].to_numpy()
blue_col = df['blue'].to_numpy()
green_col = df['green'].to_numpy()

def f(x):    
    return str(x['normalized_red'])+str(x['normalized_green'])+str(x['normalized_blue'])

hashed_df = copy.deepcopy(normalized_df)
hashed_df['hash'] = hashed_df.apply(f, axis=1)
hashed_df

,name,red,green,blue,alpha,id,normalized_red,normalized_green,normalized_blue,hash
0,Graphics,26,127,81,255,0,0.10196,0.49804,0.31765,0.101960.498040.31765
1,Intercom,128,29,137,255,0,0.50196,0.11373,0.53725,0.501960.113730.53725
2,Lamp1,8,10,16,255,11,0.03137,0.03922,0.06275,0.031370.039220.06275
3,AC_Duct,80,184,210,255,0,0.31373,0.72157,0.82353,0.313730.721570.82353
4,Graffiti3,12,112,165,255,19,0.04706,0.43922,0.64706,0.047060.439220.64706
...,...,...,...,...,...,...,...,...,...,...
372,Tunnel_Service_Door_LOD0,67,53,173,255,19,0.26275,0.20784,0.67843,0.262750.207840.67843
373,MetalPillar2_SM,25,72,203,255,19,0.09804,0.28235,0.79608,0.098040.282350.79608
374,Vent_Smoke,138,98,4,255,0,0.54118,0.38431,0.01569,0.541180.384310.01569
375,SMPL_Female_Autonomous,35,213,234,255,20,0.13725,0.83529,0.91765,0.137250.835290.91765


In [48]:
# Generate table from color to id.
def label_from_color(color):
    norm_r = round(color[0], 5)
    norm_g = round(color[1], 5)
    norm_b = round(color[2], 5)
    # TODO(Toni): can be greatly optimized... TO SLOW NOW
    # TODO(Toni): you are comparing floats with == ......
#     print(normalized_df.loc[(normalized_df['normalized_red'] == norm_r) & 
#                   (normalized_df['normalized_green'] == norm_g) & 
#                   (normalized_df['normalized_blue'] == norm_b)])
    label_list = normalized_df.loc[(normalized_df['normalized_red'] == norm_r) & 
                  (normalized_df['normalized_green'] == norm_g) & 
                  (normalized_df['normalized_blue'] == norm_b)]['id'].unique().tolist()
    return label_list[0]

In [49]:
def calc_corresp(est_pcl, gt_pcl, correspondences):
    total_negative_matches = 0
    total_positive_matches = 0
    total_correspondences = len(correspondences)
    count = 0
    for correspondence in correspondences:
        est_label_id = label_from_color(est_pcl.colors[correspondence[0]])
        gt_label_id = label_from_color(gt_pcl.colors[correspondence[1]])
        if est_label_id == gt_label_id:
            total_positive_matches += 1
        else: 
            total_negative_matches += 1
        count += 1
#         if np.allclose(est_pcl.colors[correspondence[0]],
#                        gt_pcl.colors[correspondence[1]]):
#             total_positive_matches += 1
#         else:
#             total_negative_matches += 1

    print("Positive matches: ",total_positive_matches)
    print("Negative matches: ", total_negative_matches)
    print("Total correspondences: ", total_correspondences)
    assert(total_correspondences == total_negative_matches + total_positive_matches)
    print ("Positive: {}  % ".format(total_positive_matches / total_correspondences * 100))
    print ("Negative: {}  % ".format(total_negative_matches / total_correspondences * 100))

In [50]:
calc_corresp(est_pcl, gt_pcl, correspondences)

Positive color matches:  5961670
Negative color matches:  244560
Total correspondences:  6206230
Positive: 96.05944349468196  % 
Negative: 3.9405565053180434  % 


In [25]:
print(est_pcl.colors[0])

[0.57254902 0.70980392 0.37647059]


In [42]:
# Compare labels between correspondences:
total_label_correspondences = {i:0 for i in normalized_df['id'].unique()}
print(total_label_correspondences)
# Initialize dictionaries to 0:
total_positive_matches = {i:0 for i in normalized_df['id'].unique()}
total_negative_matches = copy.deepcopy(total_positive_matches)

{0: 0, 11: 0, 19: 0, 4: 0, 18: 0, 10: 0, 3: 0, 14: 0, 1: 0, 7: 0, 6: 0, 15: 0, 8: 0, 20: 0}


In [37]:
def calc_per_label_corresp():
    for correspondence in correspondences:
        est_pcl_color = est_pcl.colors[correspondence[0]]
        color_hash = str(est_pcl_color[0])+str(est_pcl_color[1])+str(est_pcl_color[2])
        label_id = label_from_color(est_pcl_color)
        if np.allclose(est_pcl_color, gt_pcl.colors[correspondence[1]]):
            total_positive_matches[label_id] += 1
        else:
            total_negative_matches[label_id] += 1

In [38]:
calc_per_label_corresp()

In [41]:
print("Dict for total_positive_matches: \n", total_positive_matches)
print("Dict for total_negative_matches: \n", total_negative_matches)
for label_id in df['id'].unique().tolist():
    total_label_correspondences[label_id] = (total_positive_matches[label_id] + total_negative_matches[label_id])
print("Dict for total_label_correspondences: \n", total_label_correspondences)
positive_match = 0
negative_match = 0
correspondences = 0
for label_id in df['id'].unique().tolist():
    positive_match += total_positive_matches[label_id]
    negative_match += total_negative_matches[label_id]
    correspondences += total_label_correspondences[label_id]
    if (total_label_correspondences[label_id] > 0):
        print ("ID: {}".format(label_id))
        print ("Positive: {}  % ".format(total_positive_matches[label_id] / total_label_correspondences[label_id] * 100))
        print ("Negative: {}  % ".format(total_negative_matches[label_id] / total_label_correspondences[label_id] * 100))
print("Summary Positive label matches: {} % ".format(positive_match / correspondences * 100))
print("Summary Negative label matches: {} % ".format(negative_match / correspondences * 100))

Dict for total_positive_matches: 
 {0: 0, 11: 0, 19: 4947, 4: 0, 18: 0, 10: 2165, 3: 44071, 14: 1858, 1: 0, 7: 10489, 6: 4186, 15: 918, 8: 0, 20: 0}
Dict for total_negative_matches: 
 {0: 420691, 11: 118628, 19: 1250337, 4: 1954055, 18: 16096, 10: 372, 3: 1796593, 14: 60801, 1: 52579, 7: 102786, 6: 3841, 15: 360817, 8: 0, 20: 0}
Dict for total_label_correspondences: 
 {0: 420691, 11: 118628, 19: 1255284, 4: 1954055, 18: 16096, 10: 2537, 3: 1840664, 14: 62659, 1: 52579, 7: 113275, 6: 8027, 15: 361735, 8: 0, 20: 0}
ID: 0
Positive: 0.0  % 
Negative: 100.0  % 
ID: 11
Positive: 0.0  % 
Negative: 100.0  % 
ID: 19
Positive: 0.39409408548185115  % 
Negative: 99.60590591451815  % 
ID: 4
Positive: 0.0  % 
Negative: 100.0  % 
ID: 18
Positive: 0.0  % 
Negative: 100.0  % 
ID: 10
Positive: 85.33701221915648  % 
Negative: 14.662987780843517  % 
ID: 3
Positive: 2.394299013834138  % 
Negative: 97.60570098616587  % 
ID: 14
Positive: 2.965256387749565  % 
Negative: 97.03474361225044  % 
ID: 1
Positive: 0